# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = "Weather.csv"
weather_df = pd.read_csv(data_file)
weather_df.head()

,City,Country,Latitude,longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Alofi,NU,-19.06,-169.92,78.80,94,84,6.93,1603019794
1,Cherskiy,RU,68.75,161.30,20.84,92,77,9.95,1603019794
2,Saint Paul Harbor,US,57.79,-152.41,37.40,81,75,3.36,1603019794
3,Bilma,NE,18.69,12.92,98.76,8,0,2.77,1603019794
4,Saint-Philippe,RE,-21.36,55.77,82.40,61,90,17.22,1603019794


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[["Latitude", "longitude"]]
humidity = weather_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                         (weather_df["Temperature"] >= 50) & (weather_df["Temperature"] <= 75)]
city_df.dropna(inplace = True) 
city_df

C:\Users\TEHS\Anaconda3\envs\homework\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,City,Country,Latitude,longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
20,Port Lincoln,AU,-34.73,135.87,52.39,72,0,9.22,1603019797
55,Vrangel',RU,42.73,133.08,53.24,53,0,2.15,1603019805
125,Kars,TR,40.60,43.09,69.80,18,0,5.82,1603019815
205,Los Andes,CL,-32.83,-70.60,60.01,99,0,0.40,1603019828
265,Buin,CL,-33.73,-70.75,64.00,93,0,2.24,1603019838
312,Corrales,US,35.24,-106.61,53.01,21,0,1.99,1603019688
443,Maciel,AR,-32.46,-60.89,75.00,88,0,4.70,1603019869
467,Constitución,CL,-35.33,-72.42,51.42,83,0,5.93,1603019873


In [6]:
# Hotel Map
# Store into variable named `hotel_df`.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [7]:
#create hote dataframe
hotel_df = city_df

#add hotel name
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

C:\Users\TEHS\Anaconda3\envs\homework\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Country,Latitude,longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
20,Port Lincoln,AU,-34.73,135.87,52.39,72,0,9.22,1603019797,
55,Vrangel',RU,42.73,133.08,53.24,53,0,2.15,1603019805,
125,Kars,TR,40.60,43.09,69.80,18,0,5.82,1603019815,
205,Los Andes,CL,-32.83,-70.60,60.01,99,0,0.40,1603019828,
265,Buin,CL,-33.73,-70.75,64.00,93,0,2.24,1603019838,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():

    # get city name, latitude and longtitude
    lat = row["Latitude"]
    lng = row["longitude"]
    city_name = row["City"]
    
    # add location to params
    params["location"] = f"{lat},{lng}"

    # make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 20: Port Lincoln.
Closest hotel in Port Lincoln is Port Lincoln Hotel.
------------
Retrieving Results for Index 55: Vrangel'.


C:\Users\TEHS\Anaconda3\envs\homework\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Closest hotel in Vrangel' is Gostinichnyy Kompleks Vostok.
------------
Retrieving Results for Index 125: Kars.
Closest hotel in Kars is Cheltikov Hotel.
------------
Retrieving Results for Index 205: Los Andes.
Closest hotel in Los Andes is Termas el Corazon.
------------
Retrieving Results for Index 265: Buin.
Closest hotel in Buin is Paihuen.
------------
Retrieving Results for Index 312: Corrales.
Closest hotel in Corrales is Days Inn by Wyndham Rio Rancho.
------------
Retrieving Results for Index 443: Maciel.
Closest hotel in Maciel is Life Hotel.
------------
Retrieving Results for Index 467: Constitución.
Closest hotel in Constitución is Bed & Breakfast Sweet Dreams.
------------
-------End of Search-------


In [9]:
# Display the hotel data frame
hotel_df

,City,Country,Latitude,longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
20,Port Lincoln,AU,-34.73,135.87,52.39,72,0,9.22,1603019797,Port Lincoln Hotel
55,Vrangel',RU,42.73,133.08,53.24,53,0,2.15,1603019805,Gostinichnyy Kompleks Vostok
125,Kars,TR,40.60,43.09,69.80,18,0,5.82,1603019815,Cheltikov Hotel
205,Los Andes,CL,-32.83,-70.60,60.01,99,0,0.40,1603019828,Termas el Corazon
265,Buin,CL,-33.73,-70.75,64.00,93,0,2.24,1603019838,Paihuen
312,Corrales,US,35.24,-106.61,53.01,21,0,1.99,1603019688,Days Inn by Wyndham Rio Rancho
443,Maciel,AR,-32.46,-60.89,75.00,88,0,4.70,1603019869,Life Hotel
467,Constitución,CL,-35.33,-72.42,51.42,83,0,5.93,1603019873,Bed & Breakfast Sweet Dreams


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "longitude"]]

In [14]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))